In [1]:
import sys
sys.path.append("../")

In [2]:
import torch

In [3]:
from prepare import pick_one_dataset, tokenized_dataset, RE_Dataset

In [4]:
import pickle as pickle
import os
import pandas as pd
import torch

# Dataset 구성.
class RE_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()
        }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# 처음 불러온 tsv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다.
# 변경한 DataFrame 형태는 baseline code description 이미지를 참고해주세요.
def preprocessing_dataset(dataset, label_type):
    label = []
    for i in dataset[8]:
        if i == "blind":
            label.append(100)
        else:
            label.append(label_type[i])
            
    out_dataset = pd.DataFrame(
        {
            "sentence": dataset[1],
            "entity_01": dataset[2],
            "entity_02": dataset[5],
            "label": label,
        }
    )
    return out_dataset


# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
    # load label_type, classes
    with open("/opt/ml/input/data/label_type.pkl", "rb") as f:
        label_type = pickle.load(f)
    # load dataset
    dataset = pd.read_csv(dataset_dir, delimiter="\t", header=None)
    # preprecessing dataset
    dataset = preprocessing_dataset(dataset, label_type)

    return dataset


# bert input을 위한 tokenizing.
# tip! 다양한 종류의 tokenizer와 special token들을 활용하는 것으로도 새로운 시도를 해볼 수 있습니다.
# baseline code에서는 2가지 부분을 활용했습니다.
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset["entity_01"], dataset["entity_02"]):
        temp = ""
        temp = e01 + "[SEP]" + e02
        concat_entity.append(temp)
        
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset["sentence"]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=100,
        add_special_tokens=True,
    )
    return tokenized_sentences

train_dataset = load_data("/opt/ml/input/data/train/train.tsv")
# dev_dataset = load_data("./dataset/train/dev.tsv")
train_label = train_dataset["label"].values
# dev_label = dev_dataset['label'].values

# tokenizing dataset
tokenized_train = tokenized_dataset(train_dataset, tokenizer)
# tokenized_dev = tokenized_dataset(dev_dataset, tokenizer)

# make dataset for pytorch.
RE_train_dataset = RE_Dataset(tokenized_train, train_label)

from torch.utils.data import DataLoader
dataloader = DataLoader(RE_train_dataset, batch_size=40, shuffle=False)

NameError: name 'tokenizer' is not defined

In [5]:
from config import get_args
from losses import get_lossfn
from prepare import load_dataloader
from networks import load_model_and_tokenizer
from optimizers import get_optimizer, get_scheduler

args = get_args()

model, tokenizer = load_model_and_tokenizer(args)  # to(args.device)
train_dataloader, test_dataloader = load_dataloader(args, tokenizer)
loss_fn = get_lossfn(args, tokenizer)
optimizer = get_optimizer(args, model)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

TypeError: get_lossfn() takes 1 positional argument but 2 were given

In [9]:
model.backbone.electra

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(35000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0): ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [6]:
tokenizer.convert_ids_to_tokens

PreTrainedTokenizerFast(name_or_path='monologg/koelectra-base-v3-discriminator', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [11]:
len(train_dataloader)

119

In [4]:
scheduler = get_scheduler(args, optimizer)

In [7]:
args.batch_size = 64
train_dataloader, test_dataloader = load_dataloader(args, tokenizer)

In [8]:
batch = next(iter(train_dataloader))

In [9]:
inputs = {
    "input_ids": batch["input_ids"].to(args.device),
    "attention_mask": batch["attention_mask"].to(args.device),
    "token_type_ids": batch["token_type_ids"].to(args.device),
}

labels = batch["labels"].to(args.device)

In [10]:
labels

tensor([ 6, 31,  5,  4, 11, 38, 10,  5, 41, 27, 32,  8, 32,  4,  7, 35, 39, 20,
        15, 38, 37, 27, 29, 13,  9, 36, 10, 24,  2,  9, 16,  4, 20, 13, 31, 18,
        19, 23, 10, 30, 23, 26, 12, 24, 16, 23, 23, 30, 16, 25, 37,  1,  8, 17,
         8, 15,  0, 41,  6, 15, 29,  2, 15, 36], device='cuda:0')

In [27]:
tokenizer

PreTrainedTokenizerFast(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [24]:
model.config.max_length

20

In [34]:
tokenizer.all_special_ids, tokenizer.all_special_tokens

([1, 3, 0, 2, 4], ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'])

In [42]:
inputs['input_ids'][0]

tensor([    2,  5157,  5814, 10801,     3, 17860,  5699,  6221,     3,  5157,
         5814, 10801,  5692, 18963, 17860,  5699,  6221,     1,  2851, 25818,
         3360, 21900,     1,  3760, 11860, 15061, 20736,  6192,  5975,  5695,
         6145, 10666,    12,    39,  6203,  6204,    13, 11145, 12958,  5668,
        11006, 13462,  5813, 17967, 12717, 12225, 10849, 11860, 19386,  3518,
        11342, 18043,    18,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [54]:
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
tokens

['[CLS]',
 '랜드',
 '##로',
 '##버',
 '[SEP]',
 '자동차',
 '[SEP]',
 '영국에서',
 '사용되는',
 '스포츠',
 '유',
 '##틸',
 '##리티',
 '자동차',
 '##의',
 '브랜드',
 '##로는',
 '랜드',
 '##로',
 '##버',
 '(',
 'L',
 '##and',
 'R',
 '##ove',
 '##r',
 ')',
 '와',
 '지',
 '##프',
 '(',
 'J',
 '##ee',
 '##p',
 ')',
 '가',
 '있으며',
 ',',
 '이',
 '브랜드',
 '##들은',
 '자동차',
 '##의',
 '종류',
 '##를',
 '일컫',
 '##는',
 '말로',
 '사용',
 '##되기도',
 '한다',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [55]:
tokens[:7]

['[CLS]', '랜드', '##로', '##버', '[SEP]', '자동차', '[SEP]']

In [66]:
print(tokens[7:53])

['영국에서', '사용되는', '스포츠', '유', '##틸', '##리티', '자동차', '##의', '브랜드', '##로는', '랜드', '##로', '##버', '(', 'L', '##and', 'R', '##ove', '##r', ')', '와', '지', '##프', '(', 'J', '##ee', '##p', ')', '가', '있으며', ',', '이', '브랜드', '##들은', '자동차', '##의', '종류', '##를', '일컫', '##는', '말로', '사용', '##되기도', '한다', '.', '[SEP]']


In [64]:
inputs['token_type_ids'][0][53]

tensor(0, device='cuda:0')

In [53]:
inputs['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')

In [69]:
inputs['attention_mask'][0][52]

tensor(1, device='cuda:0')

In [30]:
inputs['token_type_ids']

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [46]:
preds = model(**inputs)

In [55]:
torch.all(preds.logits == preds[0])

tensor(True, device='cuda:0')

In [57]:
torch.all(torch.argmax(preds.logits, dim=1) == preds.logits.argmax(-1))

tensor(True, device='cuda:0')

In [64]:
preds.logits[0]

tensor([ 0.0499,  0.0685, -0.1423, -0.0227,  0.1487,  0.0880,  0.0919,  0.0270,
        -0.1064,  0.0438, -0.0964, -0.0858, -0.1560,  0.0886, -0.1527, -0.1060,
         0.0382, -0.1262,  0.1676,  0.1987, -0.0044,  0.0146, -0.0205, -0.1747,
         0.1622, -0.1595, -0.0071, -0.0875,  0.0483,  0.0226,  0.0397, -0.0548,
         0.1366, -0.0666,  0.1584,  0.0705,  0.1497, -0.0272,  0.0443, -0.0591,
        -0.0937, -0.0366], device='cuda:0', grad_fn=<SelectBackward>)

In [5]:
batch = next(iter(train_dataloader))

In [6]:
inputs = {
"input_ids": batch["input_ids"].to(args.device),
"attention_mask": batch["attention_mask"].to(args.device),
"token_type_ids": batch["token_type_ids"].to(args.device),
}

labels = batch["labels"].to(args.device)

In [59]:
preds.logits.argmax(-1)

tensor([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19],
       device='cuda:0')

In [65]:
labels

tensor([ 0,  0,  0,  0, 10,  2, 10,  0,  0, 11,  0,  0, 10,  0,  0,  2],
       device='cuda:0')

In [97]:
model.train()

preds = model(**inputs)
loss = loss_fn(preds.logits, labels)

RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 31.72 GiB total capacity; 22.42 GiB already allocated; 10.56 MiB free; 22.56 GiB reserved in total by PyTorch)

In [96]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [98]:
optimizer.step()

In [95]:
preds = model(**inputs)

In [80]:
labels = torch.tensor([ 1,  1,  1,  1, 10,  2, 10,  1,  1, 11,  1,  1, 10,  1,  1,  2]).to(args.device)

In [78]:
labels

tensor([ 1,  1,  1,  1, 10,  2, 10,  1,  1, 11,  1,  1, 10,  1,  1,  2])

In [99]:
preds.logits.argmax(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [100]:
preds.logits

tensor([[ 4.1756e+00,  1.4531e+00,  1.3950e+00, -6.9986e-01, -6.8249e-01,
         -5.7307e-01, -7.1698e-01, -8.7550e-01, -5.6105e-01, -9.6622e-01,
          2.1145e+00,  3.9846e-01, -1.0907e+00, -9.4720e-01, -6.3393e-01,
         -5.1403e-01, -1.2511e+00, -7.9433e-01, -3.3240e-01, -1.0091e-01,
         -9.3954e-01, -2.9875e-01, -9.9866e-01, -9.7344e-01, -5.7321e-01,
         -7.2513e-01, -1.0356e+00, -9.6626e-01, -7.0563e-01, -5.2256e-01,
         -3.0522e-01, -6.8546e-01, -5.4925e-01, -6.1712e-01, -1.0708e-01,
         -5.6433e-01, -8.7626e-01, -4.8756e-01, -1.0900e+00, -3.0684e-01,
         -6.1557e-01, -5.7203e-01],
        [ 4.1710e+00,  1.6871e+00,  1.4333e+00, -7.3609e-01, -7.4050e-01,
         -6.6859e-01, -6.3897e-01, -7.9299e-01, -6.7784e-01, -8.7162e-01,
          2.1071e+00,  2.6285e-01, -7.6712e-01, -8.9290e-01, -8.0032e-01,
         -4.7125e-01, -1.1804e+00, -7.6778e-01, -2.5641e-01, -2.2592e-01,
         -1.0123e+00, -3.0076e-01, -8.4895e-01, -8.2095e-01, -6.8195e-01,
  

In [14]:
tokenizer.pad_token_id

0

In [ ]:
model.train()
epoch_loss = 0.0

for i, batch in enumerate(dataloader):
    optimizer.zero_grad()

    inputs = {
        "input_ids": batch["input_ids"].to(args.device),
        "attention_mask": batch["attention_mask"].to(args.device),
        "token_type_ids": batch["token_type_ids"].to(args.device),
    }

    labels = batch["labels"].to(args.device)

    preds = model(**inputs)
    preds = preds[0]  # preds.logits

    loss = loss_fn(preds, labels)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

return epoch_loss / len(dataloader)

In [30]:
idx = 13
print(inputs['input_ids'][idx])

tensor([   101,  23130,  17360,    102,   9022,  18778,  13764,    102,  64749,
         12424,  98068,   9638,   8896,  83685,   9420,  21386,  52951,  22879,
          9283,  20626,  12692,  23130,  17360,  11467,   8988,  52951, 119424,
         41850,    117,  65349,  10739,   9323,  51431,  11102,  18347,   9022,
         96972,   9689,  11018,   9022,  18778,  13764,  27023, 106154,   9953,
         24017,  25820,   9414,  37601,   9637,  32537,  28981,  69448,   8853,
         48549,  12490,  11664,   9379,  33323,  14102,    119,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [29]:
idx = 13
print(inputs['attention_mask'][idx], inputs['token_type_ids'][idx], sep='\n')

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
model.get_input_embeddings()

Embedding(119547, 768, padding_idx=0)

In [35]:
model.bert.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(119547, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [6]:
preds = model(**inputs)

In [4]:
tbatch = next(iter(test_dataloader))

In [56]:
scheduler.get_lr()

[5e-05, 5e-05]

In [48]:
for nd in model.named_parameters():
    print(nd[0])

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [5]:
inputs = {
"input_ids": tbatch["input_ids"].to(args.device),
"attention_mask": tbatch["attention_mask"].to(args.device),
"token_type_ids": tbatch["token_type_ids"].to(args.device),
}

labels = tbatch["labels"].to(args.device)

In [31]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'][7])

['[CLS]',
 '울',
 '##산',
 '현대',
 '##미',
 '##포',
 '##조',
 '##선',
 '돌',
 '##고',
 '##래',
 '[SEP]',
 '내',
 '##셔',
 '##널',
 '##리그',
 '[SEP]',
 '구',
 '##단',
 '##은',
 '김',
 '##호',
 '##곤',
 '감독',
 '##의',
 '후',
 '##임',
 '##으로',
 '울',
 '##산',
 '현대',
 '##미',
 '##포',
 '##조',
 '##선',
 '돌',
 '##고',
 '##래',
 '축구',
 '##단',
 '감독',
 '##으로',
 '재',
 '##임',
 '중',
 ',',
 '2',
 '##차',
 '##례',
 '##나',
 '내',
 '##셔',
 '##널',
 '##리그',
 '통',
 '##합',
 '##우',
 '##승',
 '##을',
 '이',
 '##끌',
 '##었던',
 '조',
 '##민',
 '##국',
 '감독',
 '##을',
 '울',
 '##산',
 '현대',
 '##의',
 '8',
 '##대',
 '감독',
 '##으로',
 '선',
 '##임',
 '##하였다',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '

In [25]:
inputs['input_ids'].shape

torch.Size([32, 128])

In [26]:
inputs['attention_mask'].shape

torch.Size([32, 128])

In [27]:
inputs['token_type_ids'].shape

torch.Size([32, 128])

In [28]:
inputs['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [18]:
preds = model(**inputs)

In [36]:
preds

tensor([[ 1.4807,  0.1397, -0.1768,  ...,  0.3132, -0.1463, -0.1504],
        [ 1.3067, -0.0480, -0.3804,  ...,  0.4351,  0.3172,  0.1289],
        [ 1.2167,  0.1411, -0.5035,  ...,  0.3620,  0.0765,  0.1103],
        ...,
        [ 1.7326, -0.1825, -0.3070,  ...,  0.3482,  0.3763,  0.0315],
        [ 1.1521, -0.1345, -0.1390,  ...,  0.1098,  0.2424, -0.0393],
        [ 1.1846,  0.0549, -0.0856,  ...,  0.1326,  0.3473, -0.0636]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [35]:
labels

tensor([ 4, 15,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0, 27, 15,  2,  0,  0, 10,
         0, 15,  0,  0,  0,  7,  1,  0, 10,  0,  4, 15,  0,  2],
       device='cuda:0')

In [37]:
loss_fn(preds, labels)

tensor(2.6586, device='cuda:0', grad_fn=<NllLossBackward>)

In [19]:
labels

tensor([ 4, 15,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0, 27, 15,  2,  0,  0, 10,
         0, 15,  0,  0,  0,  7,  1,  0, 10,  0,  4, 15,  0,  2],
       device='cuda:0')

In [20]:
preds.argmax(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [21]:
preds.argmax(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [22]:
args.model_name_or_path

'bert-base-multilingual-cased'

In [30]:
inputs['input_ids'][7]

tensor([   101,   9607,  21386, 104518,  22458,  55530,  20626,  18471,   9091,
         11664,  37388,    102,   8996, 119049,  49881,  44689,    102,   8908,
         24989,  10892,   8935,  20309, 118639,  81571,  10459,  10003,  36240,
         11467,   9607,  21386, 104518,  22458,  55530,  20626,  18471,   9091,
         11664,  37388,  37905,  24989,  81571,  11467,   9659,  36240,   9694,
           117,    123,  23466,  58762,  16439,   8996, 119049,  49881,  44689,
          9879,  33188,  27355,  48210,  10622,   9638, 118705,  61439,   9678,
         36553,  20479,  81571,  10622,   9607,  21386, 104518,  10459,    129,
         14423,  81571,  11467,   9428,  36240,  12609,    119,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [26]:
tokenizer.convert_ids_to_tokens(100)

'[UNK]'

In [16]:
{**inputs}

{'input_ids': tensor([[101, 100, 100,  ...,   0,   0,   0],
         [101, 100, 100,  ...,   0,   0,   0],
         [101, 100, 100,  ...,   0,   0,   0],
         ...,
         [101, 100, 100,  ...,   0,   0,   0],
         [101, 100, 100,  ...,   0,   0,   0],
         [101, 100, 100,  ...,   0,   0,   0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')}

In [12]:
inputs['input_ids'].shape

torch.Size([32, 128])

In [13]:
inputs['input_ids'].unsqueeze(0).shape

torch.Size([1, 32, 128])

In [16]:
from train import run

In [17]:
run(args, model, loss_fn, optimizer, None, train_dataloader, test_dataloader)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 